# HOME ASSIGNMENT #3: SLACK API - TO GSHEET
**Mục đích của bài Assignment**
- Lấy thông tin các Users từ Slack của DataCracy (BTC, Mentors và Learners)
- `**[Optional 1]**` Đưa danh sách Users lên Google Spreadsheet, để theo dõi 
- `**[Optional 2]**` Lấy thông tin Assignment Submission và số Reviews trên `#atom-assignmentnt2` và cập nhật lên Spreadsheet, để theo dõi các học viên đã nộp bài và được review

**Các kiến thức sẽ áp dụng**
- Ôn lại và luyện tập thêm về concept API (cụ thể sử dụng API Slack)
- Trích xuất thông tin từ JSON
- Dùng module gspread để đưa thông tin lên Google Spreadsheet

## 0. Load Modules

In [10]:
import requests #-> Để gọi API
import re #-> Để xử lý data dạng string
from datetime import datetime as dt #-> Để xử lý data dạng datetime
import gspread #-> Để update data lên Google Spreadsheet
from gspread_dataframe import set_with_dataframe #-> Để update data lên Google Spreadsheet
import pandas as pd #-> Để update data dạng bản
import json 
from oauth2client.service_account import ServiceAccountCredentials #-> Để nhập Google Spreadsheet Credentials
import os

## 1. Slack API: User List
* Bạn có thể đọc lại về concept API [HERE](https://anhdang.gitbook.io/datacracy/atom/3-data-tools-2/3.2-spotify-api-and-postman)
* Assignment này sẽ dùng Slack API để lấy thông tin về Learners và theo dõi các bài tập đã nộp và được review (sau đó cập nhật lên Google Spreadsheet)
* ===> **NOTICE**: Slack API authorize bằng Bearer Token `xoxb-...-...-...` (Sẽ được cung cấp riêng)
* Update file `env_variable.json` như trong [Assignment#2](../assignment_2/home_assignment_2.ipynb)
* ==> Nếu bạn dùng Google Colab, upload file vào Colab ([Hướng dẫn](https://colab.research.google.com/notebooks/io.ipynb))

In [11]:
!ls

datacracy-assignment3-50e5e1b38166.json  env_variable.json  sample_data


In [ ]:
with open('env_variable.json', 'r') as j:
    json_data = json.load(j)
    print(json_data)

In [13]:
## Load SLACK_BEARER_TOKEN
os.environ['SLACK_BEARER_TOKEN'] = json_data['SLACK_BEARER_TOKEN'] 

In [14]:
## Gọi API từ Endpoints (Input - Token được đưa vào Headers)
## Challenge: Thử gọi API này bằng Postman
endpoint = "https://slack.com/api/users.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response_json = requests.post(endpoint, headers=headers).json() 
user_dat = response_json['members']

### TODO #1
Hoàn tất đoạn code sau

In [15]:
## Loop qua JSON file và extract các thông tin quan trọng (id, name, display_name, real_name_normalized, title, phone, is_bot)
## Hint: Bạn có thể dùng Postman hoặc in user_dat JSON để xem cấu trúc (schema), dùng Ctrl+F để tìm các keys (id, name, display_name, real_name_normalized, title, phone, is_bot)
user_dict = {'user_id':[], 'name':[], 'display_name':[],'real_name':[],'title':[],'phone':[],'is_bot':[]}
for i in range(len(user_dat)):
  user_dict['user_id'].append(user_dat[i]['id'])
  user_dict['name'].append(user_dat[i]['name'])
  user_dict['display_name'].append(user_dat[i]['profile']['display_name'])
  user_dict['real_name'].append(user_dat[i]['profile']['real_name_normalized'])
  user_dict['title'].append(user_dat[i]['profile']['title'])
  user_dict['phone'].append(user_dat[i]['profile']['phone'])
  user_dict['is_bot'].append(user_dat[i]['is_bot'])

In [18]:
user_df = pd.DataFrame(user_dict) ## Dùng pandas để convert dictionaries thành bảng
user_df.head(5) ## Chỉ in 5 dòng đầu (chủ yếu để xem cấu trúc)

,user_id,name,display_name,real_name,title,phone,is_bot
0,USLACKBOT,slackbot,Slackbot,Slackbot,,,False
1,U01AT4T75JB,loclexuan26392,Loc Le Xuan,Loc Le Xuan,,,False
2,U01AVDY7JET,locle.ds,Loc Le Xuan,Loc Le Xuan,,,False
3,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,,False
4,U01C48T7S1J,huyenhoang.design,Thanh Huyen Hoang,Thanh Huyen Hoang,,,False


In [19]:
user_df[user_df.display_name == 'MAD'] ## Lọc thông tin của MAD, trên DataFrame (bạn có thể Google thêm)

,user_id,name,display_name,real_name,title,phone,is_bot
3,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,,False


-------------- HẾT PHẦN BẮT BUỘC ---------------------

## Option 1: Update data => Google SpreadSheet

### TODO#2
Tạo service account (output là file json), file này để cho phép ta access vào Google Spreadsheet:

1. Làm theo hướng dẫn: [Google Create a Service Account](https://support.google.com/a/answer/7378726?hl=en)
![google_service_account](https://github.com/ngocvu74/atom-assignments/blob/main/img/google_service_account.png?raw=1)
2. Lưu file JSON (chứa credential về máy)
![gservice_acc_json](https://github.com/ngocvu74/atom-assignments/blob/main/img/gservice_acc_json.png?raw=1)
3. Nhớ Enable [Google Drive API](https://console.cloud.google.com/marketplace/product/google/drive.googleapis.com?q=search&referrer=search&project=quickstart-313303) (Nếu bạn chạy code báo lỗi chưa enable API thì vào link trong phần lỗi để Enable, sau khi kích hoạt có thể cần vài phút để chạy được)
![enable_api](https://github.com/ngocvu74/atom-assignments/blob/main/img/enable_api.png?raw=1)
* ==> Upload file Gsheet Credential JSON nếu bạn dùng Colab 
* ==> Nếu bạn để key trong repo git, **NHỚ** để file json vào `.gitignore` để không bị leaked key)


In [20]:
!ls

datacracy-assignment3-50e5e1b38166.json  env_variable.json  sample_data


In [21]:
## Authorize bằng JSON
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'datacracy-assignment3-50e5e1b38166.json', scope)
gc = gspread.authorize(credentials)
print("DONE!")

DONE!


**Tạo Spreadsheet**

1. Tạo Spreadsheet trên google
2. Invite account trong `client_email` (file JSON Gsheet Credential bên trên) vào Spreadsheet (quyền Editor)
![enable_api](https://github.com/ngocvu74/atom-assignments/blob/main/img/enable_api.png?raw=1)
3. Lấy `SPREADSHEET_KEY` (nằm trong chính URL của Spreadhstee): `https://docs.google.com/spreadsheets/d/<SPREADSHEET_KEY>/edit#gid=0`

![add_gsheet](https://github.com/ngocvu74/atom-assignments/blob/main/img/add_gsheet.png?raw=1)

In [22]:
# ACCES GOOGLE SHEET
sheet_index_no = 0
spreadsheet_key = '17s9-PbcZgR4sjATEbDfDQZr_rtaQ27j0ENE3z7zzbms' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.get_worksheet(sheet_index_no) #-> 0 - first sheet, 1 - second sheet etc. 

# APPEND DATA TO SHEET
set_with_dataframe(worksheet, user_df) #-> Upload user_df vào Sheet đầu tiên trong Spreadsheet

# DONE: Bây giờ bạn có thể mở spreadsheet và kiểm tra nội dung đã update chứ

![slack_user_gsheet](https://github.com/ngocvu74/atom-assignments/blob/main/img/slack_user_gsheet.png?raw=1)

-------------- HẾT PHẦN OPTION 1 ---------------------

## Option 2: Ai đã nộp bài?


### Slack API: Channel List

In [23]:
## Gọi SLACK API để list tất cả các channel
endpoint = "https://slack.com/api/conversations.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response = requests.post(endpoint, headers=headers).json() 
channel_ls = response['channels']
print(channel_ls)

[{'id': 'C01B4PVGLVB', 'name': 'general', 'is_channel': True, 'is_group': False, 'is_im': False, 'created': 1600856703, 'is_archived': False, 'is_general': True, 'unlinked': 0, 'name_normalized': 'general', 'is_shared': False, 'parent_conversation': None, 'creator': 'U01BE2PR6LU', 'is_ext_shared': False, 'is_org_shared': False, 'shared_team_ids': ['T01B7SGGMLJ'], 'pending_shared': [], 'pending_connected_team_ids': [], 'is_pending_ext_shared': False, 'is_member': False, 'is_private': False, 'is_mpim': False, 'topic': {'value': '', 'creator': '', 'last_set': 0}, 'purpose': {'value': 'This is the one channel that will always include everyone. It’s a great spot for announcements and team-wide conversations.', 'creator': 'U01BE2PR6LU', 'last_set': 1600856703}, 'previous_names': [], 'num_members': 63}, {'id': 'C01BYH7JHB5', 'name': 'contents', 'is_channel': True, 'is_group': False, 'is_im': False, 'created': 1601882889, 'is_archived': False, 'is_general': False, 'unlinked': 0, 'name_normaliz

In [24]:
channel_ls[0] ## Thử extract record đầu tiên để xem schema  => name: general, id: C01B4PVGLVB

{'created': 1600856703,
 'creator': 'U01BE2PR6LU',
 'id': 'C01B4PVGLVB',
 'is_archived': False,
 'is_channel': True,
 'is_ext_shared': False,
 'is_general': True,
 'is_group': False,
 'is_im': False,
 'is_member': False,
 'is_mpim': False,
 'is_org_shared': False,
 'is_pending_ext_shared': False,
 'is_private': False,
 'is_shared': False,
 'name': 'general',
 'name_normalized': 'general',
 'num_members': 63,
 'parent_conversation': None,
 'pending_connected_team_ids': [],
 'pending_shared': [],
 'previous_names': [],
 'purpose': {'creator': 'U01BE2PR6LU',
  'last_set': 1600856703,
  'value': 'This is the one channel that will always include everyone. It’s a great spot for announcements and team-wide conversations.'},
 'shared_team_ids': ['T01B7SGGMLJ'],
 'topic': {'creator': '', 'last_set': 0, 'value': ''},
 'unlinked': 0}

### TODO#3 
* Tìm id của channel #atom-assignment2

### Slack API: List messages trong 1 channel

In [48]:
atom_assignment2_id = "Can't find"
for i in channel_ls:
    if i['name'] == 'atom-assignment2':
        atom_assignment2_id = i['id']

print(atom_assignment2_id)

C021FSDN7LJ


In [65]:
endpoint = "https://slack.com/api/conversations.history"
data = {"channel": "C021FSDN7LJ"} ## This is ID of assignment#2 channel
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}

In [67]:
response_json = requests.post(endpoint, data=data, headers=headers).json()
# print(response_json)
msg_ls = response_json['messages']

In [68]:
msg_ls[50]

{'subtype': 'channel_join',
 'text': '<@U01UMBW8G8J> has joined the channel',
 'ts': '1620743421.003300',
 'type': 'message',
 'user': 'U01UMBW8G8J'}

In [69]:
not_learners_id = ['U01BE2PR6LU']

In [70]:
not_learners_id = ['U01BE2PR6LU'] # -> Remove MA from the user_id
github, reply_count, reply_users_count, reply_users, latest_reply = '','','','',''
msg_dict = {'ts': [], 'user': [], 'reply_count': [], 'reply_users_count': [], 'reply_users': [], 'latest_reply': [], 'github': []}
for i in range(len(msg_ls)):
    ts = dt.fromtimestamp(float(msg_ls[i]['ts'])) # -> Convert timestamp Epoch thành dạng dễ đọc
    user = msg_ls[i]['user'] # -> Lấy thông tin người post messages
    if msg_ls[i]['user'] not in not_learners_id:
        if 'attachments' in msg_ls[i].keys():
      # print(msg_ls[i].keys())
            text = msg_ls[i]['text']
            github_link = re.findall('(?:https?://)?(?:www[.])?github[.]com/[\w-]+/?', text) #-> Submission là các message có link github
      # print(msg_ls[i])
            if len(github_link) > 0: github = github_link[0]
            if 'reply_count' in msg_ls[i].keys(): reply_count = msg_ls[i]['reply_count'] #-> Extract số review
            if 'reply_users_count' in msg_ls[i].keys(): reply_users_count = msg_ls[i]['reply_users_count']
            if 'reply_users' in msg_ls[i].keys(): reply_users = msg_ls[i]['reply_users']
            if 'latest_reply' in msg_ls[i].keys(): latest_reply = dt.fromtimestamp(float(msg_ls[i]['latest_reply']))
            msg_dict['ts'].append(ts)
            msg_dict['user'].append(user)
            msg_dict['reply_count'].append(reply_count)
            msg_dict['reply_users_count'].append(reply_users_count)
            msg_dict['reply_users'].append(reply_users)
            msg_dict['latest_reply'].append(latest_reply)
            msg_dict['github'].append(github)

In [71]:
msg_df = pd.DataFrame(msg_dict)
msg_df.head(5)


,ts,user,reply_count,reply_users_count,reply_users,latest_reply,github
0,2021-05-17 00:28:18.005500,U01UTGRMGEQ,,,,,https://github.com/nguyenhonghanh2511/
1,2021-05-15 02:01:57.003500,U01UJ9M3P1T,,,,,https://github.com/cnhhoang850/
2,2021-05-14 23:54:46.001200,U01UJ9P1S3X,,,,,https://github.com/lethuthao1368/
3,2021-05-14 18:14:27.022900,U01UTGVPE7N,,,,,https://github.com/hoaintp/
4,2021-05-14 16:57:51.019900,U01UMJ451V1,3,3,"[U01UMJ451V1, U01UMC0N5U2, U01UMJ36QD9]",2021-05-19 14:32:40.004800,https://github.com/annaho124/


### TODO#4
* Tạo thành 1 bảng chứa các thông tin trên và update lên Spreadsheet (Sheet: Assignment#2 Submission)

In [72]:
## Authorize bằng JSON
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'datacracy-assignment3-50e5e1b38166.json', scope)
gc = gspread.authorize(credentials)
print("DONE!")

DONE!


In [73]:
# ACCES GOOGLE SHEET
sheet_index_no = 1
spreadsheet_key = '17s9-PbcZgR4sjATEbDfDQZr_rtaQ27j0ENE3z7zzbms' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.get_worksheet(sheet_index_no) #-> 0 - first sheet, 1 - second sheet etc. 

# APPEND DATA TO SHEET
set_with_dataframe(worksheet, msg_df) #-> Upload user_df vào Sheet thứ 2 trong Spreadsheet

-------------- HẾT PHẦN OPTION 2 ---------------------